In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
# Thank you

## Import Canadian Permanent Residence Score Draw from IRCC website

In [39]:
#import libraries
import requests
import json
from html import unescape
from bs4 import BeautifulSoup
import pandas as pd
from pyspark.sql import Row
from pyspark.sql.types import IntegerType

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 41, Finished, Available)

In [40]:
#make a request to the json file on the website
domain_name = 'www.canada.ca'
json_file = '/content/dam/ircc/documents/json/ee_rounds_123_en.json'

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 42, Finished, Available)

In [57]:
#define delta file path 
delta_table_path = 'Tables/CanadaPRScores'

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 59, Finished, Available)

In [42]:
#combine domain name with json file 
url = 'http://' + domain_name + json_file
print(url)

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 44, Finished, Available)

http://www.canada.ca/content/dam/ircc/documents/json/ee_rounds_123_en.json


In [43]:
#retrieve data from the json
response = requests.get(url)


StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 45, Finished, Available)

In [44]:
#parse the JSON response text 
data = json.loads(response.text)

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 46, Finished, Available)

In [45]:
#retrieve url from data url in json file
def geturl(urltag):
    urltag = unescape(urltag)
    soup = BeautifulSoup(urltag, 'html.parser')
    #find the a tag
    a = soup.find('a')
    href = a['href']
    urllink = url = 'http://' + domain_name + href
    return urllink


StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 47, Finished, Available)

In [46]:
PRscores = []
#Extract the items from data
for item in data['rounds']:
    prdic = {}
    urltag = item['drawNumberURL']
    prdic['URL_of_Draw_Details'] = geturl(urltag)
    prdic['Draw_Date'] = item['drawDateFull']
    prdic['Round_Type'] = item['drawName']
    prdic['Invitations_Issued'] = item['drawSize']
    prdic['CRS_Cutoff_Score'] = item['drawCRS']
    PRscores.append(prdic)

print(len(PRscores))

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 48, Finished, Available)

288


In [47]:
#print first and last item to view items 
print(f'first item is {PRscores[0]}')
print('\n')
print(f'last item is {PRscores[-1]}')

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 49, Finished, Available)

first item is {'URL_of_Draw_Details': 'http://www.canada.ca/content/canadasite/en/immigration-refugees-citizenship/corporate/mandate/policies-operational-instructions-agreements/ministerial-instructions/express-entry-rounds/invitations.html?q=287', 'Draw_Date': 'February 29, 2024', 'Round_Type': 'French language proficiency (2024-1)', 'Invitations_Issued': '2,500', 'CRS_Cutoff_Score': '336'}


last item is {'URL_of_Draw_Details': 'http://www.canada.ca/en/immigration-refugees-citizenship/corporate/mandate/policies-operational-instructions-agreements/ministerial-instructions/express-entry-rounds/invitations-1.html', 'Draw_Date': 'January 31, 2015', 'Round_Type': 'No Program Specified', 'Invitations_Issued': '779', 'CRS_Cutoff_Score': '886'}


In [48]:
#convert into a dataframe
df = pd.DataFrame(PRscores)
df

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 50, Finished, Available)

,URL_of_Draw_Details,Draw_Date,Round_Type,Invitations_Issued,CRS_Cutoff_Score
0,http://www.canada.ca/content/canadasite/en/imm...,"February 29, 2024",French language proficiency (2024-1),"2,500",336
1,http://www.canada.ca/content/canadasite/en/imm...,"February 28, 2024",General,"1,470",534
2,http://www.canada.ca/content/canadasite/en/imm...,"February 16, 2024",Agriculture and agri-food occupations (2023-1),150,437
3,http://www.canada.ca/content/canadasite/en/imm...,"February 14, 2024",Healthcare occupations (2023-1),"3,500",422
4,http://www.canada.ca/content/canadasite/en/imm...,"February 13, 2024",General,"1,490",535
...,...,...,...,...,...
283,http://www.canada.ca/en/immigration-refugees-c...,"March 20, 2015",No Program Specified,"1,620",481
284,http://www.canada.ca/en/immigration-refugees-c...,"February 27, 2015",No Program Specified,"1,187",735
285,http://www.canada.ca/en/immigration-refugees-c...,"February 20, 2015",Canadian Experience Class,849,808
286,http://www.canada.ca/en/immigration-refugees-c...,"February 7, 2015",No Program Specified,779,818


In [50]:
#data cleaning 
#convert draw date to datetime 
df['Draw_Date'] = pd.to_datetime(df['Draw_Date'])
#convert all no program specfied to general
df['Round_Type'] = df['Round_Type'].replace('No Program Specified','General')
#convert draw size and draw crs score to numbers
df['Invitations_Issued'] = pd.to_numeric(df['Invitations_Issued'].str.replace(',', '')) 
df['CRS_Cutoff_Score'] = pd.to_numeric(df['CRS_Cutoff_Score'].str.replace(',', '')) 

print(df.dtypes)

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 52, Finished, Available)

URL_of_Draw_Details            object
Draw_Date              datetime64[ns]
Round_Type                     object
Invitations_Issued              int64
CRS_Cutoff_Score                int64
dtype: object


In [53]:
sparkdf = spark.createDataFrame(df)
display(sparkdf)

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 55, Finished, Available)

SynapseWidget(Synapse.DataFrame, 91d15c91-c69a-49ee-aa50-e0f9da3c505e)

In [54]:
#check the types 
sparkdf.printSchema()

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 56, Finished, Available)

root
 |-- URL_of_Draw_Details: string (nullable = true)
 |-- Draw_Date: timestamp (nullable = true)
 |-- Round_Type: string (nullable = true)
 |-- Invitations_Issued: long (nullable = true)
 |-- CRS_Cutoff_Score: long (nullable = true)



In [55]:

# Convert 'Invitations Issue' and 'CRS Score of Lowest-Ranked Candidate Invited' to integer
sparkdf = sparkdf.withColumn('Invitations_Issued', sparkdf['Invitations_Issued'].cast(IntegerType()))
sparkdf = sparkdf.withColumn('CRS_Cutoff_Score', sparkdf['CRS_Cutoff_Score'].cast(IntegerType()))

sparkdf.printSchema()

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 57, Finished, Available)

root
 |-- URL_of_Draw_Details: string (nullable = true)
 |-- Draw_Date: timestamp (nullable = true)
 |-- Round_Type: string (nullable = true)
 |-- Invitations_Issued: integer (nullable = true)
 |-- CRS_Cutoff_Score: integer (nullable = true)



In [58]:
#write dataframe into lakehouse
sparkdf.write.format("delta").mode("overwrite").save(delta_table_path)

StatementMeta(, 0ccf8e41-c1b1-455f-b36a-c6aecffd7550, 60, Finished, Available)